In [1]:
%load_ext autoreload
%autoreload 2

# MLflow Classification Recipe Notebook

This notebook runs the MLflow Classification Recipe on Databricks and inspects its results. For more information about the MLflow Classification Recipe, including usage examples, see the [Classification Recipe overview documentation](https://mlflow.org/docs/latest/recipes.html#classification-recipe) the [Classification Recipe API documentation](https://mlflow.org/docs/latest/python_api/mlflow.recipes.html#module-mlflow.recipes.classification.v1.recipe).

In [2]:
from mlflow.recipes import Recipe

r = Recipe(profile="local")
r.clean()

2023/04/12 16:23:36 INFO mlflow.recipes.recipe: Creating MLflow Recipe 'mlflow-recipes-titanic' with profile: 'local'


In [3]:
# Display the DAGs (Directed Acyclic Graph)
# r.inspect()

In [4]:
r.run("ingest")

2023/04/12 16:23:38 INFO mlflow.recipes.step: Running step ingest...


In [5]:
r.run("split")

2023/04/12 16:23:39 INFO mlflow.recipes.utils.execution: ingest: No changes. Skipping.


Run MLFlow Recipe step: split
2023/04/12 16:23:40 INFO mlflow.recipes.step: Running step split...


In [6]:
training_data = r.get_artifact("training_data")
training_data.describe()

,Survived,Pclass,SibSp,Parch,Fare,Age
count,712.000000,712.000000,712.000000,712.000000,712.000000,570.000000
mean,0.386236,2.318820,0.544944,0.376404,31.362476,29.475877
std,0.487228,0.830694,1.142801,0.782297,47.619466,14.689461
min,0.000000,1.000000,0.000000,0.000000,0.000000,0.670000
25%,0.000000,2.000000,0.000000,0.000000,7.895800,20.000000
50%,0.000000,3.000000,0.000000,0.000000,14.454200,28.000000
75%,1.000000,3.000000,1.000000,0.000000,30.695800,38.000000
max,1.000000,3.000000,8.000000,6.000000,512.329200,80.000000


In [7]:
r.run("transform")

2023/04/12 16:23:41 INFO mlflow.recipes.utils.execution: ingest, split: No changes. Skipping.


Run MLFlow Recipe step: transform
2023/04/12 16:23:42 INFO mlflow.recipes.step: Running step transform...


Name,Type
Survived,int64
Pclass,int64
Sex,object
SibSp,int64
Parch,int64
Fare,float64
Age,float64
Embarked,object
Name,Type
onehot__Pclass_1,float64


In [8]:
r.run("train")

2023/04/12 16:23:43 INFO mlflow.recipes.utils.execution: ingest, split, transform: No changes. Skipping.


Run MLFlow Recipe step: train
2023/04/12 16:23:44 INFO mlflow.recipes.step: Running step train...
2023/04/12 16:23:45 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2023/04/12 16:23:45 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2023/04/12 16:23:45 INFO mlflow.recipes.steps.train: Training data has less than 5000 rows, skipping rebalancing.

  0%|          | 0/5 [00:00<?, ?trial/s, best loss=?]2023/04/12 16:23:51 INFO mlflow.models.evaluation.base: Evaluating the model with the default evaluator.

2023/04/12 16:23:51 INFO mlflow.models.evaluation.default_evaluator: The evaluation dataset is inferred as binary dataset, positive label is 1, negative label is 0.


 20%|██        | 1/5 [00:06<00:27,  6.93s

Metric,training,validation
accuracy_score,0.810393,0.721519
example_count,712,79
f1_score,0.743833,0.576923
false_negatives,79,11
false_positives,56,11
log_loss,0.431264,0.50216
precision_recall_auc,0.838185,0.736431
precision_score,0.777778,0.576923
recall_score,0.712727,0.576923
roc_auc,0.86166,0.814224


In [9]:
trained_model = r.get_artifact("model")
print(trained_model)

2023/04/12 16:24:21 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2023/04/12 16:24:21 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


mlflow.pyfunc.loaded_model:
  artifact_path: train/model
  flavor: mlflow.pyfunc.model
  run_id: 4977a1dfd83e42bf986ca1d7a27cfc4a



In [10]:
r.run("evaluate")

2023/04/12 16:24:21 INFO mlflow.recipes.utils.execution: ingest, split, transform, train: No changes. Skipping.


Run MLFlow Recipe step: evaluate
2023/04/12 16:24:22 INFO mlflow.recipes.step: Running step evaluate...
2023/04/12 16:24:22 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2023/04/12 16:24:22 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2023/04/12 16:24:23 INFO mlflow.models.evaluation.base: Evaluating the model with the default evaluator.
2023/04/12 16:24:23 INFO mlflow.models.evaluation.default_evaluator: The evaluation dataset is inferred as binary dataset, positive label is 1, negative label is 0.
2023/04/12 16:24:26 INFO mlflow.models.evaluation.default_evaluator: Shap explainer _PatchedKernelExplainer is used.

100%|██████████| 10/10 [00:00<00:00, 31.08it/s]
elementwise comparison failed; returning

Metric,validation,test
accuracy_score,0.721519,0.790000
example_count,79,100.000000
f1_score,0.576923,0.746988
false_negatives,11,10.000000
false_positives,11,11.000000
log_loss,0.50216,0.469276
precision_recall_auc,0.736431,0.827481
precision_score,0.576923,0.738095
recall_score,0.576923,0.756098
roc_auc,0.814224,0.846631


In [11]:
r.run("register")

2023/04/12 16:24:32 INFO mlflow.recipes.utils.execution: ingest, split, transform, train, evaluate: No changes. Skipping.


Run MLFlow Recipe step: register
2023/04/12 16:24:33 INFO mlflow.recipes.step: Running step register...
2023/04/12 16:24:33 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2023/04/12 16:24:33 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2023/04/12 16:24:33 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2023/04/12 16:24:33 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
Registered model 'titanic-local' already exists. Creating a new version of this model...
2023/04/12 16:24:33 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for 

In [12]:
# Only works with Spark
# r.run("ingest_scoring")
# r.run("predict")